In [1]:
import os, torch
import numpy as np
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import scipy
from scipy.stats import linregress, describe
from tqdm import tqdm
import matplotlib.pyplot as plt

/home/edherron/.conda/envs/diffusion/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Model class - pretrained regressor
class JF_Net(nn.Module):
	def __init__(self):
		super(JF_Net, self).__init__()
		#in_channel, out_channel, kernel_size, stride, padding=0 (default)
		self.conv1 = nn.Conv2d(1, 16, 9, 1) 
		self.bn1 = nn.BatchNorm2d(16)

		self.conv2 = nn.Conv2d(16, 32, 9, 1)
		self.bn2 = nn.BatchNorm2d(32)

		self.dropout1 = nn.Dropout2d(0.3)

		self.fc1 = nn.Linear(21632, 1024)
		self.fc4 = nn.Linear(1024, 2)

	def forward(self, x):

		h = F.relu(self.conv1(x))
		h = F.max_pool2d(h, kernel_size=2)
		h = self.bn1(h)

		h = F.relu(self.conv2(h))
		h = F.max_pool2d(h, kernel_size=2)
		h = self.bn2(h)

		h = torch.flatten(h, start_dim=1) #flatten input of [bs, c, w, h], so from dim=1
		h = F.relu(self.fc1(h))
		h = self.dropout1(h)
		h = F.relu(self.fc4(h))
		h = h.squeeze() #squeezing to reduce dims from (64,1) to (64,) to match target
		output = h
		return output

In [3]:
# Dataset class
#loads dataset and convert to a iterable for JF data
class NumpyDataset(data.Dataset):
    def __init__(self, path):
        super().__init__()
        self.data = np.load(path, allow_pickle=True)
        self.transform = transforms.Compose([transforms.ToTensor()])

    def __getitem__(self, idx):
        return (self.transform(np.float32(self.data[idx,0])),
        		 np.float32(self.data[idx,1]),
        		 np.float32(self.data[idx,2]))

    def __len__(self):
        return self.data.shape[0]

#returns data loader
def get_dataloader(path, batchsize):
    ds = NumpyDataset(path)
    dl = torch.utils.data.DataLoader(ds, batch_size=batchsize, drop_last=True, shuffle=True)
    return dl

In [4]:
data_path = '/data/Joshua/DARPA_data/augmented_JF_filtered_norm_test.npy'
test_loader = get_dataloader(path='/data/Joshua/DARPA_data/augmented_JF_filtered_norm_test.npy', batchsize=32)

In [5]:
# validation loop
def eval_loop(val_dataloader, model, device):
    model.eval()
    J_preds, J_targets, ff_preds, ff_targets = [], [], [], []
    for idx, (data, J, ff) in enumerate(val_dataloader):
        data, J, ff = data.to(device), J.to(device), ff.to(device)

        pred_j, pred_ff = model(data)

        J_preds.append(pred_j)
        J_targets.append(J)
        ff_preds.append(pred_ff)
        ff_targets.append(ff)

    return J_preds, J_targets, ff_preds, ff_targets

In [6]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = JF_Net()
model.load_state_dict(torch.load('/data/EthanHerron/InvNets/TargetedMicrostructure/targeted_microstructure_Invnet/highFidelitiy_test_run/run_001/regressor.pt'))
model.eval()
model.to(device)

FileNotFoundError: [Errno 2] No such file or directory: '/data/EthanHerron/InvNets/TargetedMicrostructure/targeted_microstructure_Invnet/highFidelitiy_test_run/run_001/regressor.pt'

In [ ]:
J_pred, J_true, ff_pred, ff_true = eval_loop(test_loader, model, device)
J_pred = np.asarray(J_pred).astype('float32')
J_true = np.asarray(J_true).astype('float32')
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(J_true, J_pred)
plt.figure(figsize=(10,10))
a = plt.axes(aspect='equal')
plt.scatter(J_true, J_pred)
plt.xlabel('J-True Values')
plt.ylabel('J-Predictions')
lims = [0, max(J_pred)+1]
plt.xlim(lims)
plt.ylim(lims)
plt.plot(J_true, (intercept + slope*J_true[:]), label = "R-squared for fill: %f" % r_value**2)
plt.legend()
plt.plot(lims, lims)
# plt.savefig(os.path.join(argsp.model_dir, 'J_R2_plot.jpg'))

ff_pred = np.asarray(ff_pred).astype('float32')
ff_true = np.asarray(ff_true).astype('float32')
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(ff_true, ff_pred)
plt.figure(figsize=(10,10))
a = plt.axes(aspect='equal')
plt.scatter(ff_true, ff_pred)
plt.xlabel('$FF$-True Values')
plt.ylabel('$FF$-Predictions')
lims = [0, max(ff_pred)+1]
plt.xlim(lims)
plt.ylim(lims)
plt.plot(J_true, (intercept + slope*J_true[:]), label = "R-squared for fill: %f" % r_value**2)
plt.legend()
plt.plot(lims, lims)
# plt.savefig(os.path.join(argsp.model_dir, 'FF_R2_plot.jpg'))